In [1]:
import os
import cv2
import numpy as np
from scipy.ndimage import interpolation as inter
from prettytable import PrettyTable
import matplotlib.pyplot as plt

In [8]:
def rotate_align(inp_image, max_angle=5, delta=0.05):
    """find angle where histogram score difference is maximum and rotate image to that angle. 
    return angle and rotated image"""
    def find_hist_score(bin_img, angle):
        """find histogram score for an image for a given angle"""
        
        rotated_img = inter.rotate(bin_img, angle, reshape=False, order=0)
        verticle_sum = np.sum(rotated_img, axis=1)
        score = np.sum((verticle_sum[1:] - verticle_sum[:-1]) ** 2)
        return verticle_sum, score

    bw_img = cv2.cvtColor(inp_image, cv2.COLOR_BGR2GRAY) ## color to gray
    bin_img = cv2.threshold(bw_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] ## binarize inp_image

    hist_score_list = []
    list_angles = np.arange(-max_angle, max_angle + delta, delta)
    
#     max_score = -999
    for angle in list_angles:
        verticle_sum, score = find_hist_score(bin_img, angle)
        hist_score_list.append(score)

    best_angle = list_angles[hist_score_list.index(max(hist_score_list))]

    height, width = inp_image.shape[:2]
    mid_pt = (width // 2, height // 2)
    M = cv2.getRotationMatrix2D(mid_pt, best_angle, 1.0)
    aligned_img = cv2.warpAffine(inp_image, M, (width, height), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return best_angle, aligned_img

In [9]:
if __name__ == '__main__':
    aligned_path = '../data/aligned_images_new/'
    misaligned_path = '../data/misaligned_images/'
    
    orig_imgs = os.listdir(misaligned_path)

    table = PrettyTable(['FileName', 'Aligned?'])
    
    if not os.path.exists(aligned_path):
        os.mkdir(aligned_path)
        
    for i, img_id in enumerate(orig_imgs):
        image = cv2.imread(os.path.join(misaligned_path, img_id))
        angle, rotated = rotate_align(image)
        if angle==0:
            table.add_row([img_id, "Already Aligned"])
        else:
            table.add_row([img_id, "Misaligned"])
        cv2.imwrite(os.path.join(aligned_path, img_id), rotated)
    print(table)

+------------------------------------------+------------+
|                 FileName                 |  Aligned?  |
+------------------------------------------+------------+
|   0e27f636242845138158f132878a3351.jpg   | Misaligned |
|                  7.jpg                   | Misaligned |
|                  5.jpg                   | Misaligned |
|   5f228310d55c683ed29aa8c0adda7efc.jpg   | Misaligned |
|                  4.jpg                   | Misaligned |
|                  10.jpg                  | Misaligned |
| 0f97914844b786b0d2ef21339286cdf6 - 2.jpg | Misaligned |
|                  41.jpg                  | Misaligned |
|                AWT009.jpg                | Misaligned |
|                 6 2.jpg                  | Misaligned |
|                  6.jpg                   | Misaligned |
|      law-firm-invoice-template.jpg       | Misaligned |
|  billing-invoice-for-legal-services.jpg  | Misaligned |
+------------------------------------------+------------+
